In [1]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import glob
import os
# import matplotlib as mpl

In [2]:
def Unitvector(x_gauss, y_gauss):
    from sklearn import preprocessing
    
    # get the slope of the tangent
    trajectory = np.asarray([x_gauss, y_gauss])
    m = np.gradient(trajectory, axis = 1)
    m_atx = m[1]/m[0]
    
    # get the tangent vector at x = x0 + 1
    tangent_x = x_gauss+1
    tangent_y = m_atx + y_gauss
    
    # get the unit tangent vector
    u_x = []
    u_y = []

    for x,y,x0,y0 in zip(tangent_x, tangent_y, x_gauss, y_gauss):
        if np.any(np.isnan([x, y])) or np.any(np.isinf([x, y])):
            unit_x = np.nan
            unit_y = np.nan
        else:
            vector = np.asarray([x-x0, y-y0]).reshape(1,-1)
            [unit_x, unit_y] = preprocessing.normalize(vector, norm = 'l2')[0]
        u_x.append(unit_x)
        u_y.append(unit_y)
    u_x = np.asarray(u_x)
    u_y = np.asarray(u_y)
    return(u_x, u_y)


def getAngle(loc, tangent):
    cross = np.cross(tangent, loc)
    dot = np.dot(tangent, loc)
    angle = np.arctan2(cross, dot)*180/np.pi
    return(angle)

def angleCorloredTrajectory(x_gauss, y_gauss, angle):
    
    fig = plt.subplots(figsize = (5,5))
    plt.scatter(x_gauss, y_gauss, s=7 , c = angle, cmap = plt.cm.jet)
    c1 = plt.Circle((0, 0), circ_r, facecolor='grey', alpha=0.3, edgecolor='none')
    ax = plt.gca()
    ax.add_artist(c1)
    ax.set_aspect('equal', adjustable = 'datalim')
    cbar = plt.colorbar()
    return(fig)

def angleColoredSweepCurves(r_gauss, circ_r, angle):
    fig = plt.subplots(figsize = (5,5))
    t0 = np.arange(len(r_gauss))*1/100
    plt.scatter(t0, r_gauss/circ_r, s = 10,
                c = angle, cmap = plt.cm.jet)
    cbar = plt.colorbar()
    return(fig)

In [12]:
data_path = './dataFolders/Final Tracking xypts/filteredUsingThreshold/'
out_path = './dataFolders/Final Tracking xypts/AngleAndRadiusIncluded/'

In [13]:
datalist = glob.glob(data_path + '**.csv')
circ_parameters = pd.read_csv('./dataFolders/circle_parameters.csv')

In [19]:
for path in datalist:
    file = pd.read_csv(path)
    x = file.tracked_x
    y = file.tracked_y
    
    name = [n for n in circ_parameters.name if n in path][0]

    circ_x = circ_parameters.loc[circ_parameters.name == name, 'circ_x'].values
    circ_y = circ_parameters.loc[circ_parameters.name == name, 'circ_y'].values
    circ_radii = circ_parameters.loc[circ_parameters.name == name, 'circ_radii'].values
    
    cent_x = x - circ_x
    cent_y = y - circ_y
    
    x_interpl = cent_x.interpolate(method='polynomial', order=3)
    y_interpl = cent_y.interpolate(method='polynomial', order=3)
    
    r = np.linalg.norm([x_interpl, y_interpl], axis = 0)
    r = r/circ_radii
    
    win_gauss = signal.gaussian(10,3)
    x_gauss = signal.convolve(x_interpl, win_gauss, 
                                  mode='same', method = 'direct') / win_gauss.sum()
    y_gauss = signal.convolve(y_interpl, win_gauss, 
                                  mode = 'same', method = 'direct') / win_gauss.sum()
    
    r_gauss = np.linalg.norm([x_gauss, y_gauss], axis = 0)
    r_normalized = r_gauss/circ_radii
    
#     df1 = pd.Series(data = x_gauss, name = 'x_gauss')
#     df2 = pd.Series(data = y_gauss, name = 'y_gauss')
#     df3 = pd.Series(data = r, name = 'radial distance')
#     df4 = pd.Series(data = r_normalized, name = 'radial distance gaussian')
    
    
    #calculate the unit tangent vectors
    u_x, u_y = Unitvector(x_gauss, y_gauss)

    angle = []
    for x0, y0, x, y in zip(x_gauss, y_gauss, u_x, u_y):
        loc = [x0, y0]
        tangent = [x, y]
        a = getAngle(loc, tangent)
        angle.append(a)

    angle = np.absolute(angle)
    for i,a in enumerate(angle):
        if a > 90:
            a = 180 - a
            angle[i] = a
            
#     df5 = pd.Series(data = angle, name = 'angle')
    
    new_file = pd.DataFrame({'cent_x': cent_x, 'cent_y': cent_y,
                            'interpol_x': x_interpl, 'interpol_y': y_interpl, 'radius_interpol': r,
                            'x_gauss': x_gauss, 'y_gauss': y_gauss, 'r_gauss': r_normalized,
                            'angle': angle})
#     new_file = pd.concat([file, df1, df2, df3, df4, df5], axis = 1)
    _,tail = os.path.split(path)
    new_path = out_path + tail
    new_file.to_csv(new_path)

C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [20]:
new_file

,cent_x,cent_y,interpol_x,interpol_y,radius_interpol,x_gauss,y_gauss,r_gauss,angle
0,-7.193107,203.983627,-7.193107,203.983627,2.758249,-5.427340,100.740889,1.363338,0.713062
1,-9.947305,202.044231,-9.947305,202.044231,2.733634,-7.354414,129.778527,1.756578,0.119682
2,-13.468409,199.369887,-13.468409,199.369887,2.700328,-8.605138,154.817471,2.095357,1.162126
3,-12.265791,198.002952,-12.265791,198.002952,2.680845,-8.897934,173.557630,2.348454,3.587671
4,-16.554140,202.375383,-16.554140,202.375383,2.743937,-8.258564,185.234409,2.505654,10.224099
5,-13.133728,201.386093,-13.133728,201.386093,2.727215,-6.671687,190.085677,2.570307,65.554756
6,-0.033471,182.088462,-0.033471,182.088462,2.460655,-4.368627,183.628306,2.482166,15.511886
7,7.789188,169.258102,7.789188,169.258102,2.289692,-2.226683,175.432216,2.370897,10.859056
8,7.352006,163.216797,7.352006,163.216797,2.207869,-0.749113,165.973893,2.242913,4.641619
9,7.216760,153.124691,7.216760,153.124691,2.071549,-0.516998,155.490492,2.101234,2.694195


In [8]:
r_gauss

NameError: name 'r_gauss' is not defined